# Clean Stanford Meter Data
Code author: Sahar H. El Abbadi
Date started: 2022-03-11
Date last edited: 2022-03-11

## Data description:
Stanford metering data, cleaned according to data processing pipeline. Includes 30-s, 60-s, and 90-s average flow rates, percent methane, and whole gas measurements. Each measurement is provided for the three different timestamp metrics: Stanford timestamps, as observed from the field; operator reported timestamps; and FlightRadar timestamps.

## Script objective:
Select data for the desired timestamp and time average. Make column names machine-readable.

In [ ]:
# Setup

# Imports
from methods_source import load_summary_files, make_operator_meter_dataset


In [ ]:
# Load data
cm_meter_raw, ghg_meter_raw, kairos_meter_raw, mair_meter_raw, sciav_meter_raw = load_summary_files()

timekeepers = ['flightradar', 'stanford', 'team']
operators = ['Carbon Mapper', 'GHGSat', 'Kairos', 'MethaneAir']

# Dictionary of raw meter data
meter_raw = {
    'Carbon Mapper': cm_meter_raw,
    'GHGSat': ghg_meter_raw,
    'Kairos': kairos_meter_raw,
    'MethaneAir': mair_meter_raw,
    'Scientific Aviation': sciav_meter_raw,
}

# Generate all datasets and save them
for timekeeper in timekeepers:
    for operator in operators:
        if operator == 'Scientific Aviation':
            pass # Not applicable for ScieAv
        else:
            make_operator_meter_dataset(operator, meter_raw[operator], timekeeper)


cm_meter = make_operator_meter_dataset('Carbon Mapper', cm_meter_raw, 'flightradar')
ghg_meter = make_operator_meter_dataset('GHGSat', ghg_meter_raw, 'flightradar')
kairos_meter = make_operator_meter_dataset('Kairos', kairos_meter_raw, 'flightradar')
mair_meter = make_operator_meter_dataset('Methane Air', mair_meter_raw, 'flightradar')



In [ ]:
import pandas as pd
import numpy as np
import pathlib

# For Scientific Aviation, we'll use their start and end times reported by operator as this is a more accurate reflection of when they were conducting measurements

sciav_meter = pd.DataFrame()
overpass_id = np.linspace(1, 18, 18)
sciav_meter['overpass_id'] = overpass_id
# datetime column set to start time to keep column heading consistent with how we handle the other operator dataframes
sciav_meter['datetime_utc'] = sciav_meter_raw['start_using_sciav']
# Create a new column specifically for measurement start and end times
sciav_meter['start_measurement'] = sciav_meter_raw['start_using_sciav']
sciav_meter['start_measurement'] = sciav_meter_raw['end_using_sciav']



In [ ]:
# Update to include new gas composition data

# Load gas comp
import pandas as pd
import datetime

def select_methane_fraction(input_datetime, gas_comp_source):
    """Determines the methane mole fraction for a given datetime.
    Inputs:
      - input_datetime is a datetime object
      - gas_comp_source is either 'su_raw', 'su_normalized', or 'km' """

    # Load the clean meter data
    gas_comp = pd.read_csv(pathlib.PurePath('02_meter_data', 'gas_comp_clean_su_km.csv'),
                       nrows=15,
                       parse_dates=['start_utc', 'end_utc'])

    # Determine methane mole fraction based on date brackets
    for index, row in gas_comp.iterrows():
        if (input_datetime > row['start_utc']) and (input_datetime <= row['end_utc']):
            methane_mole_fraction = row[gas_comp_source]

    return methane_mole_fraction

test_date = datetime.datetime(2022, 10, 12, 18, 13)
print(test_date)
select_methane_fraction(test_date, 'km')

In [36]:
import pandas as pd
test = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [4, 5, 6]
})

In [37]:
print(test)

   a  b
0  1  4
1  2  5
2  3  6


In [38]:
import numpy as np
test['c'] = np.nan

In [39]:
print(test)

   a  b   c
0  1  4 NaN
1  2  5 NaN
2  3  6 NaN


In [41]:
x='a'
test[f'{x}_2'] = np.nan

In [42]:
print(test)

   a  b   c  a_2
0  1  4 NaN  NaN
1  2  5 NaN  NaN
2  3  6 NaN  NaN
